In [1]:
%matplotlib inline
import pandas as pd
import os
import numpy as np
from datetime import datetime
import glob
from matplotlib import pyplot as plt

In [2]:
duration_file = "./duration.txt"
sensor_root_dir = "./data_tran/時刻変換済みセンサデータ"

In [3]:
df = pd.read_csv(duration_file, sep="\t", header=None)
duration_df = df.rename({
    1: "start",
    2: "end",
    3: "duration",
    4: "action"
}, axis=1).drop(0, axis=1)
duration_df

,start,end,duration,action
0,00:00:07.666,00:00:08.111,00:00:00.445,振り上げ
1,00:03:34.666,00:04:35.000,00:01:00.334,階段降り
2,00:04:42.000,00:05:43.000,00:01:01.000,階段上り
3,00:06:06.444,00:07:12.999,00:01:06.555,立っている
4,00:07:14.666,00:08:15.888,00:01:01.222,走っている
5,00:08:28.555,00:09:31.333,00:01:02.778,歩いている
6,00:09:55.888,00:10:55.777,00:00:59.889,座っている
7,00:11:35.777,00:12:37.666,00:01:01.889,階段降り
8,00:12:40.555,00:13:42.333,00:01:01.778,階段上り
9,00:14:01.555,00:15:00.666,00:00:59.111,立っている


In [4]:
def to_dt(string):
    return datetime.strptime(string, "%H:%M:%S.%f")

# Test

In [5]:
sensor_a_path = os.path.join(sensor_root_dir, "Conv-sensorA-1.csv")

sensor_a_df = pd.read_csv(sensor_a_path, header=None)

sensor_a_df = sensor_a_df.rename({
    1: "ts",
    2: "x",
    3: "y",
    4: "z",
    5: "gx",
    6: "gy",
    7: "gz"
}, axis=1).drop(0, axis=1)
sensor_a_df

,ts,x,y,z,gx,gy,gz
0,18:09:11.190,121,-73,9915,95,79,-122
1,18:09:11.200,116,-63,9886,113,103,-80
2,18:09:11.210,-30,123,9852,89,85,-104
3,18:09:11.220,-147,113,9852,95,61,-110
4,18:09:11.230,-122,-58,9910,83,13,-92
...,...,...,...,...,...,...,...
121729,18:29:28.480,443,1392,9822,211,7,-116
121730,18:29:28.490,507,1392,9744,211,-24,-141
121731,18:29:28.500,472,1373,9754,174,-48,-116
121732,18:29:28.510,458,1392,9769,113,-5,-104


In [6]:
sensor_a_df["dt"] = sensor_a_df["ts"].apply(to_dt)
sensor_a_df

,ts,x,y,z,gx,gy,gz,dt
0,18:09:11.190,121,-73,9915,95,79,-122,1900-01-01 18:09:11.190
1,18:09:11.200,116,-63,9886,113,103,-80,1900-01-01 18:09:11.200
2,18:09:11.210,-30,123,9852,89,85,-104,1900-01-01 18:09:11.210
3,18:09:11.220,-147,113,9852,95,61,-110,1900-01-01 18:09:11.220
4,18:09:11.230,-122,-58,9910,83,13,-92,1900-01-01 18:09:11.230
...,...,...,...,...,...,...,...,...
121729,18:29:28.480,443,1392,9822,211,7,-116,1900-01-01 18:29:28.480
121730,18:29:28.490,507,1392,9744,211,-24,-141,1900-01-01 18:29:28.490
121731,18:29:28.500,472,1373,9754,174,-48,-116,1900-01-01 18:29:28.500
121732,18:29:28.510,458,1392,9769,113,-5,-104,1900-01-01 18:29:28.510


In [7]:
# base on excel we find that 振り上げ -> 18:09:58.850 in ts
init_camera = to_dt(duration_df[duration_df["action"] == "振り上げ"].end.values[0])
init_camera

datetime.datetime(1900, 1, 1, 0, 0, 8, 111000)

In [8]:
init_sensor = to_dt("18:09:58.850")
init_sensor

datetime.datetime(1900, 1, 1, 18, 9, 58, 850000)

In [9]:
time_delta = init_sensor - init_camera
time_delta

datetime.timedelta(seconds=65390, microseconds=739000)

In [10]:
sensor_a_df["label"] = None

for duration_row in duration_df.iterrows():
    if duration_row[1].action == "振り上げ":
        continue
        
    camera_start = to_dt(duration_row[1].start)
    camera_end = to_dt(duration_row[1].end)
    
    sensor_start = camera_start + time_delta
    sensor_end = camera_end + time_delta
    
    sensor_a_df.loc[(sensor_a_df["dt"] > sensor_start) & (sensor_a_df["dt"] < sensor_end), "label"] = duration_row[1].action

In [11]:
sensor_a_df["label"].value_counts()

歩いている    12856
立っている    12566
階段上り     12278
走っている    12233
階段降り     12222
座っている    12066
Name: label, dtype: int64

In [12]:
sensor_a_df[sensor_a_df["label"].notnull()].drop("dt", axis=1)

,ts,x,y,z,gx,gy,gz,label
25422,18:13:25.410,-7935,1954,3465,-2788,323,-3586,階段降り
25423,18:13:25.420,-7847,1998,3445,-2855,158,-3629,階段降り
25424,18:13:25.430,-7647,2051,3387,-3008,-48,-3592,階段降り
25425,18:13:25.440,-7495,2100,3309,-3124,-310,-3531,階段降り
25426,18:13:25.450,-7315,1954,3245,-3191,-548,-3421,階段降り
...,...,...,...,...,...,...,...,...
115850,18:28:29.690,-6358,6031,4749,278,-487,-196,座っている
115851,18:28:29.700,-6280,5997,4847,351,-554,-202,座っている
115852,18:28:29.710,-6265,6011,4837,400,-652,-214,座っている
115853,18:28:29.720,-6236,6085,4881,436,-749,-244,座っている



# 本番

In [13]:
sensor_path_list = list(map(lambda x: os.path.join(sensor_root_dir, f"Conv-sensor{x}-1.csv"), "ABCDE"))
sensor_path_list

['./data_tran/時刻変換済みセンサデータ/Conv-sensorA-1.csv',
 './data_tran/時刻変換済みセンサデータ/Conv-sensorB-1.csv',
 './data_tran/時刻変換済みセンサデータ/Conv-sensorC-1.csv',
 './data_tran/時刻変換済みセンサデータ/Conv-sensorD-1.csv',
 './data_tran/時刻変換済みセンサデータ/Conv-sensorE-1.csv']

In [37]:
init_sensor_list = [
    "18:09:58.850", #A
    "18:09:55.430", #B
    "18:10:00.230", #C
    "18:09:56.500", #D
    "18:10:02.210", #E
]

In [43]:
save_dir = os.path.join(sensor_root_dir, "..", "tran_labeled")
os.makedirs(save_dir, exist_ok = True)

sensor_save_path_list = list(map(lambda x: os.path.join(save_dir, f"Conv-sensor{x}-1.csv"), "ABCDE"))

In [39]:
def read_one_sensor_file(sensor_path, init_sensor_time_str):
    sensor_df = pd.read_csv(sensor_path, header=None)

    sensor_df = sensor_df.rename({
        1: "ts",
        2: "ax",
        3: "ay",
        4: "az",
        5: "gx",
        6: "gy",
        7: "gz"
    }, axis=1).drop(0, axis=1)
    
    sensor_df["dt"] = sensor_df["ts"].apply(to_dt)
    
    init_camera = to_dt(duration_df[duration_df["action"] == "振り上げ"].end.values[0])
    init_sensor = to_dt(init_sensor_time_str)
    
    time_delta = init_sensor - init_camera
    
    sensor_df["label"] = None

    for duration_row in duration_df.iterrows():
        if duration_row[1].action == "振り上げ":
            continue

        camera_start = to_dt(duration_row[1].start)
        camera_end = to_dt(duration_row[1].end)

        sensor_start = camera_start + time_delta
        sensor_end = camera_end + time_delta

        sensor_df.loc[(sensor_df["dt"] > sensor_start) & (sensor_df["dt"] < sensor_end), "label"] = duration_row[1].action
    sensor_df = sensor_df.drop("dt", axis=1)
    return sensor_df

In [40]:
def visualize_sensor_data(sensor_df):
    colors = {
        None: "black",
        "歩いている": "cyan",
        "立っている": "magenta",
        "階段上り" : "red",
        "走っている": "yellow",
        "階段降り"  : "green",
        "座っている" : "blue",
    }
    
    c=sensor_df['label'].map(colors)
    plt.scatter(sensor_df["ts"], sensor_df["az"], c=sensor_df['label'].map(colors))
#     plt.legend()
    
#     plt.tight_layout()

In [44]:

for sensor_path, init_sensor_time_str, save_path in zip(sensor_path_list, init_sensor_list, sensor_save_path_list):
    df = read_one_sensor_file(sensor_path, init_sensor_time_str)
    print(df["label"].value_counts())
    print()
#     visualize_sensor_data(df)
    df.to_csv(save_path, index=None, encoding="shift-jis")
    


歩いている    12856
立っている    12566
階段上り     12278
走っている    12233
階段降り     12222
座っている    12066
Name: label, dtype: int64
歩いている    12856
立っている    12566
階段上り     12278
走っている    12233
階段降り     12222
座っている    12066
Name: label, dtype: int64
歩いている    12856
立っている    12566
階段上り     12278
走っている    12233
階段降り     12222
座っている    12066
Name: label, dtype: int64
歩いている    12856
立っている    12566
階段上り     12278
走っている    12233
階段降り     12222
座っている    12066
Name: label, dtype: int64
歩いている    12856
立っている    12566
階段上り     12278
走っている    12233
階段降り     12222
座っている    12066
Name: label, dtype: int64
